# Protein display

## Setup

In [1]:
#| default_exp protein

In [2]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import *

from fh_commons.core import *
from fh_commons.static import *

import json
import pandas as pd

In [3]:
#| export
def AF_display():
    "client side scripts"
    return Script(src="./imports/alphafold_display.js")

In [4]:
#| export
def pdbe_molstar():
    "Headers"
    s1 = Link(rel='stylesheet', type='text/css', href='https://cdn.jsdelivr.net/npm/pdbe-molstar@3.2.0/build/pdbe-molstar-light.css')
    s2 = Script(type='text/javascript', src='https://cdn.jsdelivr.net/npm/pdbe-molstar@3.2.0/build/pdbe-molstar-plugin.js')
    s3 = Style(
        '''
          #myViewer {
            float: left;
            width: 100%;
            height: 600px;
            position: relative;
            margin: 0px;
          }
          .msp-plugin ::-webkit-scrollbar-thumb {
            background-color: #474748 !important;
            border-radius: 10px;
          }
          .msp-right-panel { 
            display: none !important; 
          }
          #protein-form {
            margin: 20px;
          }
          
        '''
    )
    return [s1, s2, s3]

In [5]:
hdrs = bootstrap_hdrs() + pdbe_molstar()

In [6]:
show(*hdrs)

In [7]:
#| eval: False
from ngrok_token import *
url = start_ngrok(token)

ngrok tunnel opened at: https://9771-3-91-16-70.ngrok-free.app


In [8]:
app,rt = fast_app(pico=False,hdrs=hdrs)
server = JupyUvi(app)

## Check output format

In [9]:
df = pd.read_parquet('data/AM.parquet')

In [10]:
@rt
def test():
    form = Form(Input(id='uniprot_id'),Button('load'),hx_get='/sdf',target_id='result')
    result = Div(id='result')
    return Div(form, result)

@rt
def sdf(uniprot_id:str):
    protein_data = df[df['uniprot'] == uniprot_id].copy()

    if protein_data.empty:
        return Div('Uniprot id not found in the phosphosite database')

    protein_data['residue'] = protein_data.site.str[1:].astype(int)
    protein_data['CDDM_kinases'] = protein_data['CDDM'].str.split(',').str[:5]
    protein_data['PSPA_kinases'] = protein_data['PSPA'].str.split(',').str[:5]
    protein_data['AM_pathogenicity'] = protein_data['AM_pathogenicity'].round(4)

    # Convert the filtered data to a list of dictionaries
    # It needs to include residue, as it uses residue number to highlight
    out = protein_data[['residue', 'AM_pathogenicity','site_seq','source','CDDM_kinases','PSPA_kinases']]

    return Container(df2html(out))

In [11]:
htmx(url,'/sdf?uniprot_id=P10398')

## Protein Display

In [16]:
#| export
AF_script = Script(
    '''
// Global state variables
let isAlphaFoldView = false;
let currentProteinId = null;
let currentSiteData = null;

// Function to load protein
function loadProtein(proteinId, siteData) {
  // Create a new plugin instance each time the function is called
  const viewerInstance = new PDBeMolstarPlugin();

  // Set options with dynamic alphafoldView
  const options = {
    customData: {
      url: `https://alphafold.ebi.ac.uk/files/AF-${proteinId}-F1-model_v1.cif`,
      format: 'cif',
    },
    alphafoldView: isAlphaFoldView,  // Use the dynamic state variable here
    bgColor: { r: 255, g: 255, b: 255 },
    hideCanvasControls: [
      'animation',
      'controlToggle',
      'controlInfo',
      'expand',
      'selection',
      'orientation',
      'zoom',
    ],
    sequencePanel: true,
    hideControls: false,
    landscape: true,
  };

  // Get element from HTML/Template to place the viewer
  const viewerContainer = document.getElementById('myViewer');

  // Render the new instance
  viewerInstance.render(viewerContainer, options);

  // Add tooltips for AM_pathogenicity scores
  viewerInstance.events.loadComplete.subscribe(() => {
    const tooltipData = siteData.map(site => {
      const tooltipContent = Object.keys(site).map(key => {
        return `${key}: ${site[key] || 'N/A'}`;
      }).join('<br>');
      return {
        residue_number: site.residue,
        tooltip: tooltipContent,
        color: { r: 255, g: 0, b: 150 },
      };
    });

    viewerInstance.visual.select({ data: tooltipData });
    viewerInstance.visual.tooltips({ data: tooltipData });
  });
}

// Toggle alphafoldView on button click
document.getElementById('AF_view').addEventListener('click', function() {
  // Toggle the value of isAlphaFoldView
  isAlphaFoldView = !isAlphaFoldView;

  // Reload the viewer with the current protein data if available
  if (currentProteinId && currentSiteData) {
    loadProtein(currentProteinId, currentSiteData);
  }
});

// Add event listener to the load protein button
document.addEventListener('DOMContentLoaded', function() {
  const loadButton = document.getElementById('load-protein-btn');
  const proteinInput = document.getElementById('uniprot_id');

  loadButton.addEventListener('click', function() {
    const proteinId = proteinInput.value.trim();
    if (proteinId) {
      fetch(`./api/protein/${proteinId}`)
        .then(response => response.json())
        .then(data => {
          if (data.error) {
            alert(data.error);
          } else {
            // Save the current protein data
            currentProteinId = proteinId;
            currentSiteData = data.site_data;
            loadProtein(proteinId, data.site_data);
          }
        })
        .catch(error => {
          console.error('Error:', error);
          alert('An error occurred while fetching protein data');
        });
    } else {
      alert('Please enter a valid Protein ID');
    }
  });

  // Load a default protein on page load
  fetch('./api/protein/P35222')
    .then(response => response.json())
    .then(data => {
      if (!data.error) {
        // Save the default protein data
        currentProteinId = 'P35222';
        currentSiteData = data.site_data;
        loadProtein('P35222', data.site_data);
      }
    });
});

    '''
)

P10398
P35222

Key listeners:

- uniprot_id: input text
- load-protein-btn: click to load new protein
- myViewer: display protein
- AF_view: control alphafold color on or off

In [17]:
@rt('/')
def get():
    form = Form(
        Input(type='text', id = 'uniprot_id', placeholder='Enter Uniprot ID (e.g., P10398)'),
        Button('Load Protein', type='button',id='load-protein-btn'),
        id='protein-form',
    )
    viewer = Div(id='myViewer')
    # script = Script(src="./imports/alphafold_display.js")
    AF_button = Button('Turn On/Off Alphafold Color', type='button',id='AF_view'),
    blank=Div(style='height: 600px;')

    return Titled('PDBe Mol* JS Plugin Demo - AlphaFold View', form, AF_button, viewer, AF_script, blank)


@rt('/api/protein/{uniprot_id}')
def get(uniprot_id: str):
    # Filter the dataframe for the given UniProt ID
    protein_data = df[df['uniprot'] == uniprot_id].copy()

    if protein_data.empty:
        return {'error': 'Uniprot id not found in the phosphosite database'}

    protein_data['residue'] = protein_data.site.str[1:].astype(int)
    protein_data['CDDM_kinases'] = protein_data['CDDM'].str.split(',').str[:5]
    protein_data['PSPA_kinases'] = protein_data['PSPA'].str.split(',').str[:5]
    protein_data['AM_pathogenicity'] = protein_data['AM_pathogenicity'].round(4)

    # Convert the filtered data to a list of dictionaries
    # It needs to include residue, as it uses residue number to highlight
    site_data = protein_data[['residue', 'AM_pathogenicity','site_seq','source','CDDM_kinases','PSPA_kinases']].to_dict('records')

    return {'site_data': site_data}

In [18]:
htmx(url,'/')

## End

In [1]:
#| hide
import nbdev; nbdev.nbdev_export()

In [14]:
server.stop()
kill_ngrok()

ngrok tunnel killed
